Работа с Google API происходит через так называемый протокол OAuth 2.0.

Протокол OAuth 2.0 используется во многих системах и позволяет значительно повысить уровень безопасности доступа к вашим данным. Конечно, это усложняет настройку доступа к данным по API, но мы изучим самый простой из возможных вариантов. 



Помимо протокола ОAuth 2.0, есть другие способы защиты данных с точки зрения пользователя, но все они имеют существенные недостатки. 

Использование логина и пароля

Какие недостатки имеет этот способ:

потеря логина и пароля равносильна публикации данных на Facebook;
при необходимости изменить пароль надо обходить все приложения и скрипты;
хранить пароль в открытом виде (например, в тексте кода) вообще идея плохая.
Некоторые системы позволяют использовать вместо пароля специально сгенерированный токен (примерно, как в API Яндекс.Метрики, только токен рассчитывается, как хэш от вашего пароля). Этот способ немного лучше, но все равно ненадежен.

Постоянный токен

(как отладочный токен у Яндекс.Метрики)

потеря токена в руках грамотного специалиста равносильна потере пароля.
Использование протокола SSH с приватным и открытым ключом

сложно управлять и заменять;
приватный ключ лучше никогда не передавать по сети, что накладывает массу ограничений;
в случае потери замена займет относительно много времени.
Протокол OAuth 2.0
С недостатками разобрались. Сформулируем требования, которые позволили бы повысить безопасность данных:

токен может быть получен только доверенным пользователем;
потеря токена не позволит кому-либо получить «много» данных, т. е. токен надо регулярно обновлять;
каждое приложение имеет свои права и набор сертификатов;
доступами приложений можно оперативно управлять.
Для всего этого был разработан протокол OAuth 2.0.

Какие принципы работы позволяют ему обезопасить ваши данные? Доступ к данным контролируется двумя сторонами: интерфейсом Google Developers Console, в котором вы создаете приложения и назначаете им права, и скриптом на вашей стороне, который от имени приложения запрашивает разрешения и данные у нужной системы (в нашем случае это будет Google Analytics).

Что делается на стороне Google Developers Console:

для доступа к данным создается приложение со своими правами на использование API систем Google;
у каждого приложения создаются свои сертификаты (credentials);
сертификаты могут использоваться для генерации токена только с определенного набора хостов и скриптов.
На стороне вашего скрипта:

для генерации токена (хотя бы в первый раз) требуется разрешение пользователя под нужным логином;
для каждого токена можно задавать свои разрешения на работу с системой;
время работы токена ограничено (обычно 1 час), для дальнейшей выгрузки данных нужен новый токен.
В следующем шаге мы пройдемся по всей процедуре генерации токена для получения данных Google Analytics. Сейчас нам важно помнить следующие особенности:

Токен для запросов к API (обозначается access_token) действует ограниченное время, обычно 1 час. Для его обновления используется refresh_token, который генерируется один раз (его можно обновлять, но не стоит делать это очень часто).
Приложение, от имени которого вы делаете запросы к Google Analytics, должно предварительно получить права на эти данные в интерфейсе Google Developers Console. Права назначаются на каждую систему API отдельно. Например, можно дать права на чтение последних видео Youtube-канала и редактирование прав пользователей вашего аккаунта в Google Analytics.
При первом получении access_token и refresh_token вы должны быть залогинены в свой Google-аккаунт, чтобы дать приложению разрешение на доступ к вашим данным.

Для работы с API сервисов Google необходимо создать приложение и дать ему права на использование данных нужного сервиса Google (в текущем модуле это будет Google Analytics).

Все примерно так, как мы делали для сервисов Яндекса. Заходим на https://console.developers.google.com под Google аккаунтом, который имеет доступ к данным Google Analytics нашего учебного сайта.

Для удобной работы с авторизацией Google нам необходима библиотека google-api-python-client. Для ее установки исполните следующий код (восклицательный знак в начале имитирует командную строку в Jupyter notebook, что необходимо для установки библиотек).

In [1]:
!pip install --upgrade google-api-python-client

    100% |████████████████████████████████| 61kB 798kB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 225kB 10.4MB/s a 0:00:01
    100% |████████████████████████████████| 71kB 5.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 9.1MB/s eta 0:00:01
  Stored in directory: /Users/kovaleva/Library/Caches/pip/wheels/98/82/0d/cfb126a5e40d487157e43fdb3332937713dde36b4d0e1e754d
Successfully built httplib2


In [5]:
!pip install oauth2client

In [9]:
import argparse
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
scope = ['https://www.googleapis.com/auth/analytics.readonly']
api_name = 'analytics'
api_version = 'v3'
client_secrets_path = 'client_secret.json'
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
flow = client.flow_from_clientsecrets(client_secrets_path, scope=scope, message=tools.message_if_missing(client_secrets_path))
storage = file.Storage(api_name + '.dat')
credentials = storage.get()
credentials = tools.run_flow(flow, storage, flags)

/Users/kovaleva/anaconda3/lib/python3.6/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access analytics.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=813523515641-9do6lgkqfndsgkvbtbcd9lgi6otkq70q.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


Наконец-то можно получить какой-нибудь отчет из Google Analytics. Сейчас для проверки выгрузим отчет по источникам трафика и браузерам с помощью библиотеки от Google.

Инициализируем запрос к API Google Analytics:

In [10]:
http = credentials.authorize(http=httplib2.Http())
service = build(api_name, api_version, http=http)

Записываем ID представления:

In [11]:
profile_id = '71639180'

Выбираем период, совместимые представления и метрики для отчета (более подробно параметры запроса разберем на следующем шаге).

Список измерений и метрик для API Google Analytics тут https://developers.google.com/analytics/devguides/reporting/core/dimsmets

Делаем запрос к API:
(Если 403, то пройдите по последней ссылке из сообщения об ошибке (вторая ссылка из двух, приведенных в тексте сообщения) и подтвердите доступ. 

In [12]:
service.data().ga().get(
    ids='ga:' + profile_id,
    start_date='2018-01-01',
    end_date='2018-01-15',
    metrics='ga:visits',
    dimensions='ga:source,ga:browser',
    sort='-ga:visits',
    filters='ga:medium==organic',
    start_index='1',
    max_results='25')\
.execute()

{'columnHeaders': [{'columnType': 'DIMENSION',
   'dataType': 'STRING',
   'name': 'ga:source'},
  {'columnType': 'DIMENSION', 'dataType': 'STRING', 'name': 'ga:browser'},
  {'columnType': 'METRIC', 'dataType': 'INTEGER', 'name': 'ga:visits'}],
 'containsSampledData': False,
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:source,ga:browser&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-01-01&end-date=2018-01-15&start-index=1&max-results=25',
 'itemsPerPage': 25,
 'kind': 'analytics#gaData',
 'nextLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:source,ga:browser&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-01-01&end-date=2018-01-15&start-index=26&max-results=25',
 'profileInfo': {'accountId': '40356496',
  'internalWebPropertyId': '69526380',
  'profileId': '71639180',
  'profileName': 'Все данные по веб-сайту',
  'tableId': 'ga:71639180',
  '

Ограничения при работе с API
Как и при выгрузке отчетов из Яндекс.Метрики, в API Google Analytics предусмотрены определенные квоты на количество запросов в единицу времени. В основном, вы будете сталкиваться с двумя из них:

50 000 запросов в день на один проект (несложно превысить при выгрузке исторических данных за большой период);
100 запросов за 100 секунд на одного пользователя.

На данном шаге мы научимся получать отчеты Google Analytics простыми веб-запросами, а в следующих блоках воспользуемся официальной библиотекой от Google. Использование веб-запросов напрямую (без использования библиотеки Google) позволит вам в случае необходимости расширить возможности готовых решений. Например, мы используем свою библиотеку для получения отчетов Google Analytics с расширенными возможностями:

быстрый вызов в одну строчку;
простое добавление готовых отчетов любой сложности;
легкое использование на серверах и компьютерах коллег, в том числе через proxy-сервер;
выгрузка данных за любой период по дням (для уменьшения сэмплирования GA);
выгрузка больших таблиц;
соблюдение лимитов;
реагирование на ошибки сети или временную недоступность сервиса.
Как вы помните из прошлого блока, токен для запросов к API Google Analytics обычно выдается на 1 час. После этого его надо обновить. Давайте посмотрим, как это сделать. Сначала задаем в скрипте параметры нашего приложения, которые импортируем из файла client_secret.json:

In [13]:
import json
import requests
from datetime import datetime, timedelta
from pprint import pprint
config = json.load( open('analytics.dat') )
client_id = config['client_id']
client_secret = config['client_secret']
refresh_token = config['refresh_token']
def update_token(client_id, client_secret, refresh_token):
    """Обновление токена для запросов к API. Возвращает токен"""    
    url_token = 'https://accounts.google.com/o/oauth2/token'
    params = { 'client_id' : client_id, 'client_secret' : client_secret, 
               'refresh_token' : refresh_token, 'grant_type' : 'refresh_token' }
    r = requests.post( url_token, data = params )  
    print('Токен выдан до {}'.format(datetime.today() + timedelta( hours = 1 )))
    return r.json()['access_token']

Обновим токен для запросов к API с помощью этой функции:

In [14]:
token = update_token(client_id, client_secret, refresh_token)

Токен выдан до 2019-04-14 20:52:53.900661


Получим теперь простой отчет Google Analytics, отправляя соответствующий запрос. Список нужных измерений и метрик можно найти на странице https://developers.google.com/analytics/devguides/reporting/core/dimsmets

Рассмотрим простой отчет: получим количество визитов и просмотров сайта по дням за определенный период. Задаем параметры запроса:

In [15]:
url = 'https://analyticsreporting.googleapis.com/v4/reports:batchGet'
profile_id = '71639180'
start_date = '2018-01-01'
end_date = '2018-01-07'
metrics = [ { 'expression': 'ga:sessions' }, { 'expression': 'ga:pageviews' } ]
dimensions = [ { 'name': 'ga:date' } ]

In [16]:
params = {
    "reportRequests": [
        {
            'viewId': profile_id,
            'dateRanges': [ { 'startDate': start_date, 'endDate': end_date } ],
            'metrics': metrics,
            'dimensions': dimensions
        } 
    ]
}

In [17]:
headers = { 'Authorization': 'Bearer ' + token }

Для получения отчета отправляем POST-запрос:

In [18]:
import requests
r = requests.post( url, json = params, headers = headers )


In [19]:
pprint(r.json())

{'reports': [{'columnHeader': {'dimensions': ['ga:date'],
                               'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
                                                                         'type': 'INTEGER'},
                                                                        {'name': 'ga:pageviews',
                                                                         'type': 'INTEGER'}]}},
              'data': {'isDataGolden': True,
                       'maximums': [{'values': ['137', '194']}],
                       'minimums': [{'values': ['71', '89']}],
                       'rowCount': 7,
                       'rows': [{'dimensions': ['20180101'],
                                 'metrics': [{'values': ['71', '89']}]},
                                {'dimensions': ['20180102'],
                                 'metrics': [{'values': ['77', '106']}]},
                                {'dimensions': ['20180103'],
                   

Когда мы установили библиотеку google-api-python-client, то, помимо авторизации, получили набор методов для получения отчетов с помощью API. Давайте построим отчет для мониторинга нагрузки на сайт по минутам:

Импортируем необходимые библиотеки:

In [20]:
from oauth2client import file
from apiclient.discovery import build
import httplib2

Указываем, что используем API Google Analytics:

In [21]:
api_name = 'analytics'
api_version = 'v3'

Загружаем файл analytics.dat, в котором лежат ключи нашего приложения и refresh_token:

In [22]:
storage = file.Storage(api_name + '.dat')
credentials = storage.get()

Формируем объект service, с помощью которого будем получать отчеты:

In [23]:
http = credentials.authorize(http=httplib2.Http())
service = build(api_name, api_version, http=http)

In [24]:
profile_id = '71639180'

ids — номер представления;
start_date — начальная дата выгрузки;
end_date — конечная дата выгрузки;
metrics — список метрик (столбцов) отчета;
dimensions — список измерений (что стоит в строках) отчета;
sort — по какому столбцу необходимо сортировать отчет;
filters — фильтры отчета (в нашем случае берем трафик из поисковиков);
start_index — с какой строчки таблицы начинать выгрузку (в API Яндекс.Метрики это был параметр offset);
max_results — сколько строчек вернуть в таблице (аналог limit в Яндекс.Метрике). В текущий момент максимальное значение 1000 за один запрос.

Итак, отправляем запрос и смотрим, что получилось:

In [25]:
data = service.data().ga().get(
    ids='ga:' + profile_id,
    start_date='2018-02-12',
    end_date='2018-02-18',
    metrics='ga:visits',
    dimensions='ga:dateHourMinute',
    sort='-ga:visits',
    filters='ga:medium==organic',
    start_index='1',
    max_results='25')\
.execute()
data

{'columnHeaders': [{'columnType': 'DIMENSION',
   'dataType': 'STRING',
   'name': 'ga:dateHourMinute'},
  {'columnType': 'METRIC', 'dataType': 'INTEGER', 'name': 'ga:visits'}],
 'containsSampledData': False,
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2018-02-18&start-index=1&max-results=25',
 'itemsPerPage': 25,
 'kind': 'analytics#gaData',
 'nextLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2018-02-18&start-index=26&max-results=25',
 'profileInfo': {'accountId': '40356496',
  'internalWebPropertyId': '69526380',
  'profileId': '71639180',
  'profileName': 'Все данные по веб-сайту',
  'tableId': 'ga:71639180',
  'webPropertyId': 'UA-40356496-1'},
 'query': {'dimensions': 'ga:dateHourMi

Давайте выведем результат первых 25 строк:

In [26]:
result = data['rows']
result

[['201802151309', '4'],
 ['201802161811', '4'],
 ['201802121822', '3'],
 ['201802131346', '3'],
 ['201802131415', '3'],
 ['201802141633', '3'],
 ['201802142155', '3'],
 ['201802161601', '3'],
 ['201802171434', '3'],
 ['201802171947', '3'],
 ['201802181933', '3'],
 ['201802120827', '2'],
 ['201802121015', '2'],
 ['201802121459', '2'],
 ['201802121735', '2'],
 ['201802121742', '2'],
 ['201802121749', '2'],
 ['201802121811', '2'],
 ['201802121857', '2'],
 ['201802122001', '2'],
 ['201802122022', '2'],
 ['201802122049', '2'],
 ['201802122057', '2'],
 ['201802122058', '2'],
 ['201802122123', '2']]

Если нужно выгрузить все строки большого отчета, то нам необходимо условие выхода из цикла построчной выгрузки. Проверим, что отдает API Google Analytics при окончании выгрузки. Поставим значение start_index = 10000:

In [27]:
data = service.data().ga().get(
    ids='ga:' + profile_id,
    start_date='2018-02-12',
    end_date='2018-02-18',
    metrics='ga:visits',
    dimensions='ga:dateHourMinute',
    sort='-ga:visits',
    filters='ga:medium==organic',
    start_index='10000',
    max_results='25')\
.execute()
data

{'columnHeaders': [{'columnType': 'DIMENSION',
   'dataType': 'STRING',
   'name': 'ga:dateHourMinute'},
  {'columnType': 'METRIC', 'dataType': 'INTEGER', 'name': 'ga:visits'}],
 'containsSampledData': False,
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2018-02-18&start-index=10000&max-results=25',
 'itemsPerPage': 25,
 'kind': 'analytics#gaData',
 'previousLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:71639180&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2018-02-18&start-index=9975&max-results=25',
 'profileInfo': {'accountId': '40356496',
  'internalWebPropertyId': '69526380',
  'profileId': '71639180',
  'profileName': 'Все данные по веб-сайту',
  'tableId': 'ga:71639180',
  'webPropertyId': 'UA-40356496-1'},
 'query': {'dimensions': 'ga:

Ключа 'rows' в этом случае нет в ответе. Это и будет признаком того, что выгрузка всех строк отчета завершена.